Я взяла четыре статьи про фигурное катание со sport.ru. Там выделены темы, например вид спорта, упоминающиеся лица и организации. Это скорее разметка именованых сущностей, чем ключевых слов, но именованые сущности тоже в какой-то мере ключевые слова. 

Статьи:
- [про допинг Камилы Валиевой](https://www.sports.ru/tribuna/blogs/dalniyles/3093830.html#supertop)
- [почему Гран-при России не собирает стадионы](https://www.sports.ru/tribuna/blogs/beznedokrutov/3093597.html)
- [про этап Гран-при в Шеффилде](https://www.sports.ru/tribuna/blogs/vsemlutz/3092815.html)
- [про дебют Софьи Муравьевой](https://www.sports.ru/tribuna/blogs/kissnotcry/3092657.html)

In [1]:
import pandas as pd

In [2]:
# тут лежат тексты и ключевые слова
df = pd.read_csv('key_corpus.csv', sep='%')

Я выделяла не только именованные сущности, но и важные коллокации (например, *тройной аксель* в статье про Софью Муравьеву) и слова (например, *скольжение* или *компоненты* там же)

## Автоматическое извлечение

### RAKE

In [3]:
import RAKE
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [4]:
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)

In [5]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

Здесь можно было бы сделать собственный minFrequency для каждого текста, но это утомительно. Я выбрала 2, потому что так на всех текстах получалось что-то осмысленное.

In [6]:
rake_list = [rake.run(normalize_text(text), 
                      maxWords=3, 
                      minFrequency=2) for text in df.texts]
rake_list = [ ', '.join([el[0] for el in rl]) for rl in rake_list]

In [7]:
df['rake'] = rake_list

### TextRank

In [8]:
from summa import keywords

In [9]:
TR_list = [keywords.keywords(normalize_text(text), 
                             language='russian', 
                             additional_stopwords=stop, 
                             scores=True) for text in df.texts]
TR_list = [ ', '.join([el[0] for el in tr]) for tr in TR_list]

In [10]:
df['textrank'] = TR_list

### keyBert

In [11]:
from keybert import KeyBERT

2022-11-18 13:13:04.517892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 13:13:04.600686: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-18 13:13:04.618992: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-18 13:13:04.915865: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [12]:
kw_model = KeyBERT('clips/mfaq')

/home/antonauna/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [13]:
KB_list = [kw_model.extract_keywords(normalize_text(text), 
                                     keyphrase_ngram_range=(1, 1), 
                                     stop_words=stop, 
                                     top_n=20) for text in df.texts]
KB_list = [ ', '.join([el[0] for el in kb]) for kb in KB_list]

In [14]:
df['keybert'] = KB_list

## Шаблоны

Я решила сначала выделить в тексте всё, что соответствует шаблону, а потом сделать пересечение ключевых слов и всего, что соответствует шаблону. 
Конечно, можно было применить шаблон непосредственно к ключевым словам, но для этого надо было пропускать через пайморфи начальные формы, полученные через пайморфи, и мне показалось это аморальным. 

In [15]:
punctuation = ',.<>!@"\/\'():;-–«»?'

In [16]:
pat1 = ['NOUN', 
        'ADJF', 
        'None']
pat2 = [['ADJF', 'NOUN'], 
        ['NOUN', 'NOUN'], 
        ['NOUN', 'ADJF'], 
        ['None', 'None']]
pat3 = [['ADJF', 'ADJF', 'NOUN'], ]

In [17]:
def mask(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        if t not in punctuation:
            p = m.parse(t)[0]
            pos = p.tag.POS
            if not pos:
                pos = 'None'
            lemmas.append([p.normal_form, str(pos), 0])
            
    good3 = []
    for i in range(len(lemmas)-2):
        if [lemmas[i][1], lemmas[i+1][1], lemmas[i+2][1]] in pat3:
            good3.append([lemmas[i][0], lemmas[i+1][0], lemmas[i+2][0]])
            lemmas[i][2], lemmas[i+1][2], lemmas[i+2][2] = 1, 1, 1

    good2 = []
    for i in range(len(lemmas)-1):
        if [lemmas[i][1], lemmas[i+1][1]] in pat2 and (lemmas[i][2] == 0 or lemmas[i+1][2] == 0):
            good2.append([lemmas[i][0], lemmas[i+1][0]])
            lemmas[i][2], lemmas[i+1][2] = 1, 1

    good1 = []
    for i in range(len(lemmas)):
        if lemmas[i][1] in pat1 and lemmas[i][2] == 0:
            good1.append(lemmas[i][0])
            lemmas[i][2] = 1
            
    good2 = [' '.join(el) for el in good2]
    good3 = [' '.join(el) for el in good3]
    good = good1 + good2 + good3

    return ', '.join(good)

In [18]:
masks = [mask(text) for text in df.texts]
df['masks'] = masks

In [19]:
mask_rake_list = [set([el[0] for el in rake.run(normalize_text(text), 
                                               maxWords=3, 
                                               minFrequency=2)]) & set(mask.split(', ')) 
                  for text, mask in zip(df.texts, df.masks)]

mask_rake_list = [ ', '.join(el) for el in mask_rake_list]
df['mask_rake'] = mask_rake_list

In [20]:
mask_TR_list = [set([el[0] for el in keywords.keywords(normalize_text(text), 
                                                         language='russian', 
                                                         additional_stopwords=stop, 
                                                         scores=True)]) & set(mask.split(', ')) 
                for text, mask in zip(df.texts, df.masks)]

mask_TR_list = [ ', '.join(el) for el in mask_TR_list]
df['mask_textrank'] = mask_TR_list

In [21]:
mask_KB_list = [set([el[0] for el in kw_model.extract_keywords(normalize_text(text), 
                                                               keyphrase_ngram_range=(1, 1), 
                                                               stop_words=stop, 
                                                               top_n=20)]) & set(mask.split(', ')) 
                for text, mask in zip(df.texts, df.masks)]

mask_KB_list = [ ', '.join(el) for el in mask_KB_list]
df['mask_keybert'] = mask_KB_list

In [22]:
# здесь я пробовала искать ключевые слова исключительно в тех сочетаниях, которые соответствуют шаблону
# (потому что я сначала неправильно поняла формулировку этого пункта)
# могу сказать что это повышает recall до 0.3 в rake и textrank, а в keybert понижает до 0.07
# а precision сильно падает до <0.1
# но это всё вне задания

# mask_rake_list = [rake.run(normalize_text(text), maxWords=3, minFrequency=2) for text in df.masks]
# mask_rake_list = [ ', '.join([el[0] for el in rl]) for rl in mask_rake_list]
# df['mask_rake'] = mask_rake_list

# mask_TR_list = [keywords.keywords(normalize_text(text), language='russian', additional_stopwords=stop, scores=True) for text in df.masks]
# mask_TR_list = [ ', '.join([el[0] for el in tr]) for tr in mask_TR_list]
# df['mask_textrank'] = mask_TR_list

# mask_KB_list = [kw_model.extract_keywords(normalize_text(text), keyphrase_ngram_range=(1, 1), stop_words=stop, top_n=20) for text in df.masks]
# mask_KB_list = [ ', '.join([el[0] for el in kb]) for kb in mask_KB_list]
# df['mask_keybert'] = mask_KB_list

Здесь я привожу ключевые слова из золотого стандарта к начальным формам, чтобы их можно было сравнивать с ключевыми словами, полученными автоматически.

In [23]:
new_standard = []
for string in df.golden_standard:
    new_string = []
    for el in string.split(', '):
        el = el.split(' ')
        new_el = []
        for word in el:
            new_el.append(m.parse(word)[0].normal_form)
        new_string.append(' '.join(new_el))
    new_standard.append(new_string)

In [24]:
new_standard = [', '.join(el) for el in new_standard]

In [25]:
df['pm_standard'] = new_standard

Сохраним финальный датафрейм

In [26]:
df.to_csv('key_corpus.csv', index=False, sep='%')

## Оценка

In [27]:
df = pd.read_csv('key_corpus.csv', sep='%')

In [28]:
from statistics import mean

In [29]:
def evaluate(column, df):
    expected = df.pm_standard # move
    observed = df[column]
    
    
    pre = []
    re = []
    fs = []
    for i in range(len(expected)):
        exp = expected[i].split(', ')
        obs = observed[i].split(', ')
        
        TP = len(set(exp) & set(obs)) + 0.0001
        FP = len(set(obs) - set(exp)) + 0.0001 # only in model
        FN = len(set(exp) - set(obs)) + 0.0001 # only in standard
        
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        Fscore = 2 * precision * recall / (precision + recall)
        
        pre.append(precision)
        re.append(recall)
        fs.append(Fscore)
    return mean(pre), mean(re), mean(fs)

In [30]:
stats = pd.DataFrame({'rake' : evaluate('rake', df), 
                      'textrank': evaluate('textrank', df),
                      'keyBert':evaluate('keybert', df),
                      'rake+mask':evaluate('mask_rake', df),
                      'textrank+mask':evaluate('mask_textrank', df),
                      'keyBert+mask':evaluate('mask_keybert', df)}, 
                     index = ['precision', 'recall', 'F-score'])

In [31]:
stats

,rake,textrank,keyBert,rake+mask,textrank+mask,keyBert+mask
precision,0.090580,0.040301,0.100004,0.132791,0.123337,0.262706
recall,0.172248,0.231183,0.139754,0.172248,0.231183,0.120523
F-score,0.117977,0.067489,0.115668,0.149814,0.156240,0.164797


Там, где мы использовали маски, выросла точность, потому что маски уменьшают количество лишних выделений. При этом полнота не изменилась, потому что ничего нового мы не вынимаем. 

В стандарте люди выделены как имя+фамилия (*Евгений Плющенко*), а модели часто ловят только фамилию, и из-за этого получаются заниженные оценки.

## Описание ошибок

- Автоматические методы хорошо выделяют названия организаций, если это аббревиатуры (*русада*), но плохо выделяют полные названия (*спортивный арбитражный суд*). Ещё они плохо выделяют имена (*Камила Валиева*), и это может решаться отдельным выделением именованых сущностей, из которых потом выбираются ключевые слова.
- Еще в выдаче есть стоп-слова, которых, видимо, не было в списке: *это*, *почему*. Можно расширить список стоп-слов.
- Есть штуки типа *плющенко софья муравьёв пропустить*, внутри которых есть ключевые слова. Мне кажется, этого можно избежать, если добавить какую-то синтаксическую разметку. 